In [1]:
from pathlib import Path
import csv
from itertools import groupby
import h5py
import numpy as np
import sklearn
from sklearn.cluster import KMeans
from sklearn.svm import SVC
# from sklearn.linear_model import LogisticRegression
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
import random
import matplotlib
from matplotlib import pyplot as plt

# Configuration & Utilities

In [2]:
random.seed(42)
COLORS = ["#1f77b4", "#ff7f0e", "#2ca02c", "#d62728", "#9467bd", "#8c564b", "#e377c2", "#7f7f7f", "#bcbd22", "#17becf"]
ANNOTATIONS_PATH = "/media/xtrem/data/experiments/nicolingua-0001-language-id/language-id-annotations/metadata.csv"
FEATURE_DIRS = [
    '/media/xtrem/data/experiments/nicolingua-0001-language-id/wav2vec_features-c',
    '/media/xtrem/data/experiments/nicolingua-0001-language-id/wav2vec_features-z',
    '/media/xtrem/data/experiments/nicolingua-0001-language-id/retrained-wav2vec_features-c',
    '/media/xtrem/data/experiments/nicolingua-0001-language-id/retrained-wav2vec_features-z'
]
RESULTS_DIR = 'results_003_lang_id_classification'
GPU_ID = 1

In [3]:
annotation_specification = {
    0: {
        'id': 0,
        'label': "maninka",
        'required_tags': set(['ct-speech', 'lng-maninka']),
        'forbidden_tags':  set(['lng-susu', 'lng-pular'])
    },
    1: {
        'id': 1,
        'label': "susu",
        'required_tags': set(['ct-speech', 'lng-susu']),
        'forbidden_tags':  set(['lng-maninka', 'lng-pular'])
    },
    2: {
        'id': 2,
        'label': "pular",
        'required_tags': set(['ct-speech', 'lng-pular']),
        'forbidden_tags':  set(['lng-susu', 'lng-maninka'])
    }
}

In [4]:
bias_category_specification = [
    {
        "category": "utterance",
        "subcategories": [
            {
                "subcategory": "verbal_nod",
                "tags": set(["utt-verbal-nod"])
            },
            {
                "subcategory": "multilingual",
                "tags": set(["utt-multi-lingual", "utt-multi-lingual-named-endity"])
            }
        ]
    },
    {
        "category": "speaker_count",
        "subcategories": [
            {
                "subcategory": "single",
                "tags": set(["spkr-single"])
            },
            {
                "subcategory": "multiple",
                "tags": set(["spkr-mult", "spkr-multi"])
            }
        ]
    },
    {
        "category": "gender",
        "subcategories": [
            {
                "subcategory": "male",
                "tags": set(["spkr-male"])
            },
            {
                "subcategory": "female",
                "tags": set(["spkr-female"])
            },
        ]
    },
    {
        "category": "language",
        "subcategories": [
            {
                "subcategory": "susu",
                "tags": set(["lng-susu"])
            },
            {
                "subcategory": "maninka",
                "tags": set(["lng-maninka"])
            },
            {
                "subcategory": "pular",
                "tags": set(["lng-pular"])
            }
        ]
    },
    {
        "category": "channel",
        "subcategories": [
            {
                "subcategory": "telephone",
                "tags": set(["ct-telephone"])
            },
            {
                "subcategory": "noise",
                "tags": set(["ct-noise"])
            },
            {
                "subcategory": "music",
                "tags": set(["ct-fg-music", "ct-tr-music", "ct-bg-music"])
            }
        ]
    }
]

flat_bias_category_specification = {}

for c in bias_category_specification:
    for sc in c['subcategories']:
        k = f"{c['category']}_{sc['subcategory']}"
        flat_bias_category_specification[k] = sc['tags']

_ = [print(f"{k}: {v}") for k,v in flat_bias_category_specification.items()]

utterance_verbal_nod: {'utt-verbal-nod'}
utterance_multilingual: {'utt-multi-lingual', 'utt-multi-lingual-named-endity'}
speaker_count_single: {'spkr-single'}
speaker_count_multiple: {'spkr-mult', 'spkr-multi'}
gender_male: {'spkr-male'}
gender_female: {'spkr-female'}
language_susu: {'lng-susu'}
language_maninka: {'lng-maninka'}
language_pular: {'lng-pular'}
channel_telephone: {'ct-telephone'}
channel_noise: {'ct-noise'}
channel_music: {'ct-fg-music', 'ct-tr-music', 'ct-bg-music'}


In [5]:
def to_user_friendly_feature_name(fv_name):
    name = fv_name \
        .replace("features-", "") \
        .replace("wav2vec_", "") \
        .replace("average", "avg") \
        .replace("timestep", "T") \
        .replace("c.", "Context") \
        .replace("z.", "Latent")
    return name

# Load annotations

In [6]:
def load_annotations(a_file_path, a_specification):
    with open(ANNOTATIONS_PATH) as f:
        reader = csv.DictReader(f)
        for row in reader:
            tag_set = set([t.strip() for t in row['tags'].split(";")])
            for label in annotation_specification.keys():
                spec = annotation_specification[label]
                if spec['required_tags'].issubset(tag_set):
                    if spec['forbidden_tags'].isdisjoint(tag_set):
                        yield row['file'], label, tag_set
                        break

data = list(load_annotations(ANNOTATIONS_PATH, annotation_specification))
audio_files, audio_labels, audio_tags = zip(*data)

## Inspect label counts

In [7]:
def inspect_label_counts():
    for label in annotation_specification:
        count = len([l for l in audio_labels if l == label])
        print("{:10} ({}): {}".format(
            annotation_specification[label]['label'],
            label, 
            count
        ))
inspect_label_counts()

maninka    (0): 114
susu       (1): 32
pular      (2): 28


## Balance data

In [8]:
count_per_class = 28
data = list(load_annotations(ANNOTATIONS_PATH, annotation_specification))
balanced_data = []
for label in annotation_specification:
    balanced_data.extend([d for d in data if d[1] == label][:count_per_class])
audio_files, audio_labels, audio_tags = zip(*balanced_data)

In [9]:
inspect_label_counts()

maninka    (0): 28
susu       (1): 28
pular      (2): 28


## Inspect bias category counts in balanded data

In [10]:
def inspect_bias_category_counts():
    for name, tags in flat_bias_category_specification.items():
        count = len([ts for ts in audio_tags if len(tags.intersection(ts))>0])
        print(name, count, "/", len(audio_tags))
            
inspect_bias_category_counts()

utterance_verbal_nod 48 / 84
utterance_multilingual 21 / 84
speaker_count_single 25 / 84
speaker_count_multiple 58 / 84
gender_male 81 / 84
gender_female 15 / 84
language_susu 28 / 84
language_maninka 28 / 84
language_pular 28 / 84
channel_telephone 27 / 84
channel_noise 21 / 84
channel_music 24 / 84


# Prepare 10 cross validation folds

In [11]:
TRAIN_PERCENT = .6
FOLD_COUNT = 10

n = len(audio_files)
n_train = int(np.ceil(n * .6))
n_test = n - n_train
all_indices = range(n)

cv_folds = {}
train_count_by_index = {i:0 for i in all_indices}
test_count_by_index = {i:0 for i in all_indices}

for fold_index in range(FOLD_COUNT):
    fold_rsampler = np.random.RandomState(seed=fold_index)
    train_index_set = set(fold_rsampler.choice(all_indices, n_train, replace=False))
    test_index_set = set(all_indices).difference(train_index_set)
        
    cv_folds[fold_index] = {
        'train_indices': sorted(list(train_index_set)),
        'test_indices': sorted(list(test_index_set)),
    }


# Load features

In [12]:
def load_features(audio_files, features_input_dir):
    id_list = []
    features_list = []

    for audio_file_name in audio_files:
        feature_file_name = audio_file_name.replace(".wav", ".h5context")
        feature_path = Path(features_input_dir) / feature_file_name
        with h5py.File(feature_path, 'r') as f:
            features_shape = f['info'][1:].astype(int)
            features = np.array(f['features'][:]).reshape(features_shape)
            # features = pool_feature_last_seq(features)
            features_list.append(features)
    return features_list

In [13]:
raw_features = {}
for feature_dir in FEATURE_DIRS:
    feature_name = Path(feature_dir).stem
    raw_features[feature_name] = load_features(audio_files, feature_dir)

## Inspect feature shapes

In [14]:
for feature_name in raw_features.keys():
    print("feature_name: {}. feature shape: {}".format(
        to_user_friendly_feature_name(feature_name),
        raw_features[feature_name][0].shape
    ))

feature_name: c. feature shape: (2998, 512)
feature_name: z. feature shape: (2998, 512)
feature_name: retrained-c. feature shape: (2998, 512)
feature_name: retrained-z. feature shape: (2998, 512)


## Extract feature vectors

In [15]:
def extract_last_timestep_features(raw_features):
    return raw_features[-1, :]

def extract_neuron_average_features(raw_features):
    return np.mean(raw_features, axis=0)

def identity(x):
    return x

feature_extractors = {
    'last_timestep': extract_last_timestep_features,
    'neuron_average': extract_neuron_average_features,
    'raw_features': identity
}

In [16]:
feature_vectors = {}
for feature_name in raw_features.keys():
    for feature_extractor_name in feature_extractors.keys():
        fv_name = f"{feature_name}__{feature_extractor_name}"
        feature_vectors[fv_name] = []
        for f in raw_features[feature_name]:
            feature_vectors[fv_name].append(
                feature_extractors[feature_extractor_name](f)
            )
            
        feature_vectors[fv_name] = np.array(feature_vectors[fv_name])

## Inspect feature vectors

In [17]:
for fvname in feature_vectors.keys():
    print(fvname, feature_vectors[fvname][0].shape)

wav2vec_features-c__last_timestep (512,)
wav2vec_features-c__neuron_average (512,)
wav2vec_features-c__raw_features (2998, 512)
wav2vec_features-z__last_timestep (512,)
wav2vec_features-z__neuron_average (512,)
wav2vec_features-z__raw_features (2998, 512)
retrained-wav2vec_features-c__last_timestep (512,)
retrained-wav2vec_features-c__neuron_average (512,)
retrained-wav2vec_features-c__raw_features (2998, 512)
retrained-wav2vec_features-z__last_timestep (512,)
retrained-wav2vec_features-z__neuron_average (512,)
retrained-wav2vec_features-z__raw_features (2998, 512)


# Classification Models

In [18]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from pytorch_model_summary import summary
from torch.utils.data import TensorDataset, DataLoader
import torch.optim as optim

In [19]:
class LangIdLogisticReg1(nn.Module):
    def __init__(self):
        super(LangIdLogisticReg1, self).__init__()
        self.lin1 = nn.Linear(in_features=512, out_features=3)
        
    def forward(self, x):
        x = torch.mean(x, dim=1)
        x = self.lin1(x)
        return x
    
    
class LangIdLogisticReg2(nn.Module):
    def __init__(self):
        super(LangIdLogisticReg2, self).__init__()
        self.drop0 = nn.Dropout(p=0.5)
        self.lin1 = nn.Linear(in_features=512, out_features=3)
        
    def forward(self, x):
        x = torch.mean(x, dim=1)
        x = self.drop0(x)
        x = self.lin1(x)
        return x
    
    
class LangIdDNN1(nn.Module):
    def __init__(self):
        super(LangIdDNN1, self).__init__()
        self.lin1 = nn.Linear(in_features=512, out_features=3)
        self.drop1 = nn.Dropout(p=0.3)
        
        self.lin2 = nn.Linear(in_features=3, out_features=8)
        self.drop2 = nn.Dropout(p=0.3)
        
        self.lin3 = nn.Linear(in_features=8, out_features=3)
        
        
        
    def forward(self, x):
        x = torch.mean(x, dim=1)
        
        x = self.lin1(x)
        x = F.elu(x)
        x = self.drop1(x)
        
        x = self.lin2(x)
        x = F.elu(x)
        x = self.drop2(x)
        
        x = self.lin3(x)
        
        return x
    
    
    
    
class LangIdDNN2(nn.Module):
    def __init__(self):
        super(LangIdDNN2, self).__init__()
        self.drop0 = nn.Dropout(p=0.5)
        self.lin1 = nn.Linear(in_features=512, out_features=3)
        self.drop1 = nn.Dropout(p=0.3)
        
        self.lin2 = nn.Linear(in_features=3, out_features=8)
        self.drop2 = nn.Dropout(p=0.3)
        
        self.lin3 = nn.Linear(in_features=8, out_features=3)
        
        
        
    def forward(self, x):
        x = torch.mean(x, dim=1)
        
        x = self.drop0(x)
        
        x = self.lin1(x)
        x = F.elu(x)
        x = self.drop1(x)
        
        x = self.lin2(x)
        x = F.elu(x)
        x = self.drop2(x)
        
        x = self.lin3(x)
        
        return x
    
    
    
class LangIdCNN1(nn.Module):
    def __init__(self):
        super(LangIdCNN1, self).__init__()
        
        self.conv1 = nn.Conv1d(in_channels=512, out_channels=1, kernel_size=3)
        self.drop1 = nn.Dropout(p=0.3)
        self.pool1 = nn.MaxPool1d(kernel_size=2, stride=2)
        
        self.conv2 = nn.Conv1d(in_channels=1, out_channels=3, kernel_size=3)
        self.drop2 = nn.Dropout(p=0.3)
        self.pool2 = nn.MaxPool1d(kernel_size=2, stride=2)
        
        self.conv3 = nn.Conv1d(in_channels=3, out_channels=3, kernel_size=3)
        self.drop3 = nn.Dropout(p=0.3)
        self.pool3 = nn.MaxPool1d(kernel_size=2, stride=2)
        
        self.conv4 = nn.Conv1d(in_channels=3, out_channels=3, kernel_size=3)
        self.pool4 = nn.MaxPool1d(kernel_size=2, stride=2)
        
    def forward(self, x):
        x = x.permute(0, 2, 1)
        
        x = self.conv1(x)
        x = F.elu(x)
        x = self.drop1(x)
        x = self.pool1(x)
        
        x = self.conv2(x)
        x = F.elu(x)
        x = self.drop2(x)
        x = self.pool2(x)
        
        x = self.conv3(x)
        x = F.elu(x)
        x = self.drop3(x)
        x = self.pool3(x)
        
        x = self.conv4(x)
        x = F.elu(x)
        x = self.pool4(x)
        
        x = torch.mean(x, dim=2)
        
        return x

    
class LangIdCNN2(nn.Module):
    def __init__(self):
        super(LangIdCNN2, self).__init__()
        self.drop0 = nn.Dropout(p=0.5)
        
        self.conv1 = nn.Conv1d(in_channels=512, out_channels=1, kernel_size=3)
        self.drop1 = nn.Dropout(p=0.3)
        self.pool1 = nn.MaxPool1d(kernel_size=2, stride=2)
        
        self.conv2 = nn.Conv1d(in_channels=1, out_channels=3, kernel_size=3)
        self.drop2 = nn.Dropout(p=0.3)
        self.pool2 = nn.MaxPool1d(kernel_size=2, stride=2)
        
        self.conv3 = nn.Conv1d(in_channels=3, out_channels=3, kernel_size=3)
        self.drop3 = nn.Dropout(p=0.3)
        self.pool3 = nn.MaxPool1d(kernel_size=2, stride=2)
        
        self.conv4 = nn.Conv1d(in_channels=3, out_channels=3, kernel_size=3)
        self.pool4 = nn.MaxPool1d(kernel_size=2, stride=2)
        
    def forward(self, x):
        x = x.permute(0, 2, 1)
        x = self.drop0(x)
        
        x = self.conv1(x)
        x = F.elu(x)
        x = self.drop1(x)
        x = self.pool1(x)
        
        x = self.conv2(x)
        x = F.elu(x)
        x = self.drop2(x)
        x = self.pool2(x)
        
        x = self.conv3(x)
        x = F.elu(x)
        x = self.drop3(x)
        x = self.pool3(x)
        
        x = self.conv4(x)
        x = F.elu(x)
        x = self.pool4(x)
        
        x = torch.mean(x, dim=2)
        
        return x
    
    
class LangIdRNN1(nn.Module):
    def __init__(self):
        super(LangIdRNN1, self).__init__()
        
        self.conv1 = nn.Conv1d(in_channels=512, out_channels=1, kernel_size=3)
        self.drop1 = nn.Dropout(p=0.3)
        self.pool1 = nn.MaxPool1d(kernel_size=2, stride=2)
        
        self.gru = nn.GRU(input_size=1, hidden_size=3, num_layers=1, batch_first=True)
        #self.drop2 = nn.Dropout(p=0.3)
        self.lin2 = nn.Linear(in_features=3, out_features=3)
        
        
        
    def forward(self, x):
        x = x.permute(0, 2, 1)
        
        x = self.conv1(x)
        x = F.elu(x)
        x = self.drop1(x)
        x = self.pool1(x)
        
        x = x.permute(0, 2, 1)
        # output, (h_n, c_n) = self.lstm(x)
        output, h_n = self.gru(x)
        x = output[:, -1, :] # [all batches, last time step, all neurons]
        x = self.lin2(x)
        
        return x
    
    
class LangIdRNN2(nn.Module):
    def __init__(self):
        super(LangIdRNN2, self).__init__()
        self.drop0 = nn.Dropout(p=0.5)
        
        self.conv1 = nn.Conv1d(in_channels=512, out_channels=1, kernel_size=3)
        self.drop1 = nn.Dropout(p=0.3)
        self.pool1 = nn.MaxPool1d(kernel_size=2, stride=2)
        
        self.gru = nn.GRU(input_size=1, hidden_size=3, num_layers=1, batch_first=True)
        #self.drop2 = nn.Dropout(p=0.3)
        self.lin2 = nn.Linear(in_features=3, out_features=3)
        
        
        
    def forward(self, x):
        x = x.permute(0, 2, 1)
        
        x = self.drop0(x)
        
        x = self.conv1(x)
        x = F.elu(x)
        x = self.drop1(x)
        x = self.pool1(x)
        
        x = x.permute(0, 2, 1)
        # output, (h_n, c_n) = self.lstm(x)
        output, h_n = self.gru(x)
        x = output[:, -1, :] # [all batches, last time step, all neurons]
        x = self.lin2(x)
        
        return x


# Train Classification Models

In [20]:
def get_data_for_fold(fold_id, feature_name, batch_size):
    train_indices = cv_folds[fold_id]['train_indices']
    test_indices = cv_folds[fold_id]['test_indices']    

    train_x = np.take(raw_features[feature_name], train_indices, axis=0)
    train_y = np.take(audio_labels, train_indices, axis=0)
    train_tags = np.take(audio_tags, train_indices, axis=0)
    train_bias_category_labels = {}
    for category, tags in flat_bias_category_specification.items():
        train_bias_category_labels[category] = [1 if len(tags.intersection(ts))>0 else 0 for ts in train_tags]

    test_x = np.take(raw_features[feature_name], test_indices, axis=0)
    test_y = np.take(audio_labels, test_indices, axis=0)
    test_tags = np.take(audio_tags, test_indices, axis=0)
    test_bias_category_labels = {}
    for category, tags in flat_bias_category_specification.items():
        test_bias_category_labels[category] = [1 if len(tags.intersection(ts))>0 else 0 for ts in test_tags]
        
    
    return train_x, train_y, test_x, test_y, train_bias_category_labels, test_bias_category_labels

    
def get_loaders_for_fold(fold_id, feature_name, batch_size):
    
    train_x, train_y, test_x, test_y, train_bias_category_labels, test_bias_category_labels = \
        get_data_for_fold(fold_id, feature_name, batch_size)
    
    
    
    train_dataset = TensorDataset(
        torch.tensor(train_x), 
        torch.tensor(train_y)
    )

    train_loader = DataLoader(train_dataset, batch_size=batch_size)

    test_dataset = TensorDataset(
        torch.tensor(test_x), 
        torch.tensor(test_y)
    )

    test_loader = DataLoader(test_dataset, batch_size=batch_size)
    
    return train_loader, test_loader, train_bias_category_labels, test_bias_category_labels


def get_predictions_for_logits(logits):
    probs = F.softmax(logits, dim=1)
    return torch.argmax(probs, dim=1)

In [21]:
def train_on_fold(model_class, fold_id, feature_name, batch_size, epochs):
    device = torch.device(f"cuda:{GPU_ID}")
    torch.manual_seed(0)
    results = {}
    
    model = model_class().to(device)

    train_loader, test_loader, train_bias_category_labels, test_bias_category_labels = get_loaders_for_fold(fold_id, feature_name, batch_size)

    print(summary(model_class(), torch.zeros((10, 2998, 512)), show_input=False))

    optimizer = optim.Adam(model.parameters(), lr=0.001)
    criterion = nn.CrossEntropyLoss()

    for epoch in range(1, epochs+1):
        model.train()
        train_loss = 0
        pred_train_classes = []
        true_train_classes = []

        for batch_idx, (x, y) in enumerate(train_loader):
            x = x.to(device)
            y = y.to(device)
            
            optimizer.zero_grad()
            outputs = model(x)
            pred_train_classes.extend(
                get_predictions_for_logits(outputs).cpu().numpy()
            )
            true_train_classes.extend(y.cpu().numpy())
            loss = criterion(outputs, y)

            loss.backward()
            optimizer.step()

            train_loss += loss.item()

        train_n = len(true_train_classes)
        
        train_loss = train_loss/len(train_loader)
        train_acc = sklearn.metrics.accuracy_score(true_train_classes, pred_train_classes)
        train_acc_by_bais_category = {
            category: sklearn.metrics.accuracy_score(true_train_classes, pred_train_classes, sample_weight=sw)
            for category, sw in train_bias_category_labels.items()
        }
        


        pred_test_classes = []
        true_test_classes = []
        model.eval()
        test_loss = 0
        for batch_idx, (x, y) in enumerate(test_loader):
            x = x.to(device)
            y = y.to(device)
            
            outputs = model(x)

            pred_test_classes.extend(
                get_predictions_for_logits(outputs).cpu().numpy()
            )

            true_test_classes.extend(y.cpu().numpy())

            loss = criterion(outputs, y)
            test_loss += loss.item()


        test_n = len(true_test_classes)
        
        test_loss = test_loss / len(test_loader)
        test_acc = sklearn.metrics.accuracy_score(true_test_classes, pred_test_classes)
        test_acc_by_bais_category = {
            category: sklearn.metrics.accuracy_score(true_test_classes, pred_test_classes, sample_weight=sw)
            for category, sw in test_bias_category_labels.items()
        }

        if epoch%10==0:
            print(f"Epoch: {epoch}. Train Loss: {train_loss:0.4}. Test Loss: {test_loss:0.4}. Train Acc: {train_acc:0.4}. Test Acc:{test_acc:0.4}")
            
        results[epoch] = {
            'epoch': epoch,
            'train_loss': train_loss,
            'test_loss': test_loss,
            'train_acc': train_acc,
            'test_acc': test_acc,
            'train_n': train_n,
            'test_n': test_n,
            
        }
        
        for c in train_acc_by_bais_category:
            results[epoch][f"train_acc_{c}"] = train_acc_by_bais_category[c]
            results[epoch][f"train_n_{c}"] = int(np.sum(train_bias_category_labels[c]))
            
        for c in test_acc_by_bais_category:
            results[epoch][f"test_acc_{c}"] = test_acc_by_bais_category[c]
            results[epoch][f"test_n_{c}"] = int(np.sum(test_bias_category_labels[c]))
        
    del model
    return results
    

In [22]:
import csv
from pathlib import Path

def save_results(model_name, all_results):
    for result_entry in all_results:
        feature_name = result_entry['feature_name']
        fold_index = result_entry['fold_index']
        
        Path(RESULTS_DIR).mkdir(exist_ok=True)
        fname = f"{RESULTS_DIR}/{model_name}/{feature_name}_{fold_index}.csv"
        Path(fname).parent.mkdir(parents=True, exist_ok=True)
        with open(fname, 'w') as f:
            fieldnames = sorted(result_entry['epochs'][1].keys())
            writer = csv.DictWriter(f, fieldnames=fieldnames)
            
            writer.writeheader()
            
            for epoch in sorted(result_entry['epochs'].keys()):

                writer.writerow(result_entry['epochs'][epoch])

In [ ]:
model_classes = [
    LangIdLogisticReg1, 
    LangIdLogisticReg2, 
    LangIdDNN1, 
    LangIdDNN2, 
    LangIdCNN1, 
    LangIdCNN2, 
    LangIdRNN1,
    LangIdRNN2
]

all_results = []

for fold_index in cv_folds:
    for feature_name in raw_features:
        for model_class in model_classes:
            print(f"{model_class.__name__} using {feature_name} on fold#{fold_index}")
            resutls = train_on_fold(model_class, fold_index, feature_name, batch_size=100, epochs=1000)
            all_results.append({
                'fold_index': fold_index,
                'feature_name': feature_name,
                'epochs': resutls
            })
            save_results(model_class.__name__, all_results)

LangIdLogisticReg1 using wav2vec_features-c on fold#0
-----------------------------------------------------------------------
      Layer (type)        Output Shape         Param #     Tr. Param #
          Linear-1             [10, 3]           1,539           1,539
Total params: 1,539
Trainable params: 1,539
Non-trainable params: 0
-----------------------------------------------------------------------
Epoch: 10. Train Loss: 1.073. Test Loss: 1.085. Train Acc: 0.5294. Test Acc:0.4242
Epoch: 20. Train Loss: 1.047. Test Loss: 1.07. Train Acc: 0.7059. Test Acc:0.5152
Epoch: 30. Train Loss: 1.024. Test Loss: 1.058. Train Acc: 0.6863. Test Acc:0.5455
Epoch: 40. Train Loss: 1.001. Test Loss: 1.047. Train Acc: 0.7255. Test Acc:0.5758
Epoch: 50. Train Loss: 0.9796. Test Loss: 1.037. Train Acc: 0.7255. Test Acc:0.5758
Epoch: 60. Train Loss: 0.9596. Test Loss: 1.028. Train Acc: 0.7647. Test Acc:0.6061
Epoch: 70. Train Loss: 0.9406. Test Loss: 1.021. Train Acc: 0.7647. Test Acc:0.6061
Epoch: 80

Epoch: 920. Train Loss: 0.3703. Test Loss: 1.096. Train Acc: 0.9216. Test Acc:0.5455
Epoch: 930. Train Loss: 0.3672. Test Loss: 1.098. Train Acc: 0.9216. Test Acc:0.5455
Epoch: 940. Train Loss: 0.3641. Test Loss: 1.099. Train Acc: 0.9216. Test Acc:0.5455
Epoch: 950. Train Loss: 0.361. Test Loss: 1.101. Train Acc: 0.9412. Test Acc:0.5455
Epoch: 960. Train Loss: 0.358. Test Loss: 1.102. Train Acc: 0.9412. Test Acc:0.5455
Epoch: 970. Train Loss: 0.355. Test Loss: 1.104. Train Acc: 0.9412. Test Acc:0.5455
Epoch: 980. Train Loss: 0.3521. Test Loss: 1.105. Train Acc: 0.9412. Test Acc:0.5455
Epoch: 990. Train Loss: 0.3492. Test Loss: 1.107. Train Acc: 0.9412. Test Acc:0.5455
Epoch: 1000. Train Loss: 0.3463. Test Loss: 1.108. Train Acc: 0.9412. Test Acc:0.5455
LangIdLogisticReg2 using wav2vec_features-c on fold#0
-----------------------------------------------------------------------
      Layer (type)        Output Shape         Param #     Tr. Param #
         Dropout-1           [10, 512]  

Epoch: 810. Train Loss: 0.6812. Test Loss: 0.9748. Train Acc: 0.7451. Test Acc:0.5455
Epoch: 820. Train Loss: 0.7795. Test Loss: 0.974. Train Acc: 0.7059. Test Acc:0.5455
Epoch: 830. Train Loss: 0.7091. Test Loss: 0.9714. Train Acc: 0.8039. Test Acc:0.5455
Epoch: 840. Train Loss: 0.797. Test Loss: 0.9729. Train Acc: 0.6471. Test Acc:0.5455
Epoch: 850. Train Loss: 0.7168. Test Loss: 0.9735. Train Acc: 0.7255. Test Acc:0.5455
Epoch: 860. Train Loss: 0.7818. Test Loss: 0.9694. Train Acc: 0.6471. Test Acc:0.5455
Epoch: 870. Train Loss: 0.6779. Test Loss: 0.97. Train Acc: 0.7451. Test Acc:0.5455
Epoch: 880. Train Loss: 0.7445. Test Loss: 0.9721. Train Acc: 0.7451. Test Acc:0.5455
Epoch: 890. Train Loss: 0.7762. Test Loss: 0.9726. Train Acc: 0.6471. Test Acc:0.5455
Epoch: 900. Train Loss: 0.6884. Test Loss: 0.9742. Train Acc: 0.7059. Test Acc:0.5455
Epoch: 910. Train Loss: 0.7065. Test Loss: 0.9743. Train Acc: 0.7059. Test Acc:0.5455
Epoch: 920. Train Loss: 0.6882. Test Loss: 0.9769. Train A

Epoch: 680. Train Loss: 0.4201. Test Loss: 0.9812. Train Acc: 0.902. Test Acc:0.6061
Epoch: 690. Train Loss: 0.49. Test Loss: 0.9961. Train Acc: 0.7843. Test Acc:0.6061
Epoch: 700. Train Loss: 0.3939. Test Loss: 1.004. Train Acc: 0.8431. Test Acc:0.6061
Epoch: 710. Train Loss: 0.3972. Test Loss: 0.9874. Train Acc: 0.8824. Test Acc:0.6061
Epoch: 720. Train Loss: 0.4401. Test Loss: 1.014. Train Acc: 0.8627. Test Acc:0.6061
Epoch: 730. Train Loss: 0.4676. Test Loss: 1.006. Train Acc: 0.8627. Test Acc:0.6061
Epoch: 740. Train Loss: 0.4443. Test Loss: 1.001. Train Acc: 0.8235. Test Acc:0.5455
Epoch: 750. Train Loss: 0.42. Test Loss: 1.022. Train Acc: 0.8627. Test Acc:0.6364
Epoch: 760. Train Loss: 0.2947. Test Loss: 1.025. Train Acc: 0.9608. Test Acc:0.5758
Epoch: 770. Train Loss: 0.4857. Test Loss: 1.009. Train Acc: 0.8235. Test Acc:0.5758
Epoch: 780. Train Loss: 0.349. Test Loss: 1.025. Train Acc: 0.902. Test Acc:0.6061
Epoch: 790. Train Loss: 0.3598. Test Loss: 1.021. Train Acc: 0.8824. 

Epoch: 550. Train Loss: 0.7844. Test Loss: 0.9992. Train Acc: 0.6471. Test Acc:0.4242
Epoch: 560. Train Loss: 0.876. Test Loss: 1.017. Train Acc: 0.6078. Test Acc:0.3939
Epoch: 570. Train Loss: 0.8766. Test Loss: 0.9962. Train Acc: 0.6078. Test Acc:0.4242
Epoch: 580. Train Loss: 0.787. Test Loss: 1.007. Train Acc: 0.6863. Test Acc:0.4545
Epoch: 590. Train Loss: 0.869. Test Loss: 1.003. Train Acc: 0.6275. Test Acc:0.4242
Epoch: 600. Train Loss: 0.9192. Test Loss: 0.9962. Train Acc: 0.5686. Test Acc:0.4242
Epoch: 610. Train Loss: 0.8475. Test Loss: 0.9932. Train Acc: 0.6275. Test Acc:0.4242
Epoch: 620. Train Loss: 0.892. Test Loss: 0.9975. Train Acc: 0.5686. Test Acc:0.4545
Epoch: 630. Train Loss: 0.7919. Test Loss: 0.9956. Train Acc: 0.549. Test Acc:0.3939
Epoch: 640. Train Loss: 0.8062. Test Loss: 0.9871. Train Acc: 0.6078. Test Acc:0.4545
Epoch: 650. Train Loss: 0.8732. Test Loss: 0.9919. Train Acc: 0.4902. Test Acc:0.3939
Epoch: 660. Train Loss: 0.94. Test Loss: 0.9921. Train Acc: 0.

...

Epoch: 10. Train Loss: 1.083. Test Loss: 1.146. Train Acc: 0.3922. Test Acc:0.2424
Epoch: 20. Train Loss: 1.082. Test Loss: 1.152. Train Acc: 0.3922. Test Acc:0.2424
Epoch: 30. Train Loss: 1.079. Test Loss: 1.157. Train Acc: 0.3922. Test Acc:0.2424
Epoch: 40. Train Loss: 1.075. Test Loss: 1.161. Train Acc: 0.3922. Test Acc:0.2424
Epoch: 50. Train Loss: 1.068. Test Loss: 1.162. Train Acc: 0.4706. Test Acc:0.2424
Epoch: 60. Train Loss: 1.057. Test Loss: 1.164. Train Acc: 0.5098. Test Acc:0.2424
Epoch: 70. Train Loss: 1.043. Test Loss: 1.166. Train Acc: 0.5294. Test Acc:0.2424
Epoch: 80. Train Loss: 1.024. Test Loss: 1.17. Train Acc: 0.5294. Test Acc:0.1818
Epoch: 90. Train Loss: 1.006. Test Loss: 1.176. Train Acc: 0.549. Test Acc:0.2424
Epoch: 100. Train Loss: 0.9855. Test Loss: 1.186. Train Acc: 0.549. Test Acc:0.2424
Epoch: 110. Train Loss: 0.9578. Test Loss: 1.2. Train Acc: 0.549. Test Acc:0.303
Epoch: 120. Train Loss: 0.94. Test Loss: 1.215. Train Acc: 0.549. Test Acc:0.303
Epoch: 13

Epoch: 990. Train Loss: 0.4568. Test Loss: 2.857. Train Acc: 0.7451. Test Acc:0.3333
Epoch: 1000. Train Loss: 0.4535. Test Loss: 2.604. Train Acc: 0.7451. Test Acc:0.3636
LangIdRNN1 using wav2vec_features-z on fold#4
---------------------------------------------------------------------------------
      Layer (type)                  Output Shape         Param #     Tr. Param #
          Conv1d-1                 [10, 1, 2996]           1,537           1,537
         Dropout-2                 [10, 1, 2996]               0               0
       MaxPool1d-3                 [10, 1, 1498]               0               0
             GRU-4     [10, 1498, 3], [1, 10, 3]              54              54
          Linear-5                       [10, 3]              12              12
Total params: 1,603
Trainable params: 1,603
Non-trainable params: 0
---------------------------------------------------------------------------------
Epoch: 10. Train Loss: 1.15. Test Loss: 1.128. Train Acc: 0.3529.

Epoch: 860. Train Loss: 0.1123. Test Loss: 1.52. Train Acc: 1.0. Test Acc:0.3636
Epoch: 870. Train Loss: 0.1144. Test Loss: 1.597. Train Acc: 1.0. Test Acc:0.3939
Epoch: 880. Train Loss: 0.1183. Test Loss: 1.618. Train Acc: 1.0. Test Acc:0.3636
Epoch: 890. Train Loss: 0.1155. Test Loss: 1.513. Train Acc: 1.0. Test Acc:0.4242
Epoch: 900. Train Loss: 0.11. Test Loss: 1.546. Train Acc: 1.0. Test Acc:0.3939
Epoch: 910. Train Loss: 0.1866. Test Loss: 1.89. Train Acc: 0.9608. Test Acc:0.303
Epoch: 920. Train Loss: 0.1002. Test Loss: 1.504. Train Acc: 1.0. Test Acc:0.4545
Epoch: 930. Train Loss: 0.1181. Test Loss: 1.793. Train Acc: 0.9804. Test Acc:0.3333
Epoch: 940. Train Loss: 0.09308. Test Loss: 1.638. Train Acc: 1.0. Test Acc:0.3333
Epoch: 950. Train Loss: 0.1094. Test Loss: 1.705. Train Acc: 1.0. Test Acc:0.3333
Epoch: 960. Train Loss: 0.09297. Test Loss: 1.644. Train Acc: 1.0. Test Acc:0.303
Epoch: 970. Train Loss: 0.1043. Test Loss: 1.648. Train Acc: 1.0. Test Acc:0.303
Epoch: 980. Tra

Epoch: 720. Train Loss: 0.2942. Test Loss: 1.24. Train Acc: 0.9412. Test Acc:0.4545
Epoch: 730. Train Loss: 0.3317. Test Loss: 1.283. Train Acc: 0.8824. Test Acc:0.3939
Epoch: 740. Train Loss: 0.2804. Test Loss: 1.221. Train Acc: 0.9608. Test Acc:0.4242
Epoch: 750. Train Loss: 0.3082. Test Loss: 1.24. Train Acc: 0.9804. Test Acc:0.3939
Epoch: 760. Train Loss: 0.2857. Test Loss: 1.208. Train Acc: 0.9412. Test Acc:0.4545
Epoch: 770. Train Loss: 0.2351. Test Loss: 1.222. Train Acc: 0.9608. Test Acc:0.4242
Epoch: 780. Train Loss: 0.2623. Test Loss: 1.373. Train Acc: 0.9608. Test Acc:0.3939
Epoch: 790. Train Loss: 0.2382. Test Loss: 1.242. Train Acc: 0.9804. Test Acc:0.5152
Epoch: 800. Train Loss: 0.3064. Test Loss: 1.223. Train Acc: 0.8824. Test Acc:0.4848
Epoch: 810. Train Loss: 0.2102. Test Loss: 1.279. Train Acc: 0.9608. Test Acc:0.3636
Epoch: 820. Train Loss: 0.2403. Test Loss: 1.231. Train Acc: 0.9412. Test Acc:0.4545
Epoch: 830. Train Loss: 0.2889. Test Loss: 1.27. Train Acc: 0.902. 

Epoch: 630. Train Loss: 0.7215. Test Loss: 0.9026. Train Acc: 0.8824. Test Acc:0.6061
Epoch: 640. Train Loss: 0.7175. Test Loss: 0.9004. Train Acc: 0.8824. Test Acc:0.6364
Epoch: 650. Train Loss: 0.7135. Test Loss: 0.8982. Train Acc: 0.8824. Test Acc:0.6364
Epoch: 660. Train Loss: 0.7095. Test Loss: 0.8961. Train Acc: 0.8824. Test Acc:0.6667
Epoch: 670. Train Loss: 0.7056. Test Loss: 0.894. Train Acc: 0.8824. Test Acc:0.6667
Epoch: 680. Train Loss: 0.7017. Test Loss: 0.8919. Train Acc: 0.8824. Test Acc:0.6667
Epoch: 690. Train Loss: 0.6978. Test Loss: 0.8898. Train Acc: 0.8824. Test Acc:0.6667
Epoch: 700. Train Loss: 0.694. Test Loss: 0.8878. Train Acc: 0.8824. Test Acc:0.6667
Epoch: 710. Train Loss: 0.6902. Test Loss: 0.8858. Train Acc: 0.8824. Test Acc:0.6667
Epoch: 720. Train Loss: 0.6865. Test Loss: 0.8838. Train Acc: 0.8824. Test Acc:0.6667
Epoch: 730. Train Loss: 0.6827. Test Loss: 0.8819. Train Acc: 0.8824. Test Acc:0.6667
